# Implementing a Key-Value Database

The goal of this project is to implement a key-value store class called KVStore using a b-tree. The BTree class was created during the lessons in the unit on tree data structures, and can be found in the btree.py file. 

## Creating the Init and Add Methods

In [45]:
from btree import BTree

In [46]:
class KVStore(BTree):
    def __init__(self):
        super().__init__(split_threshold=2)
        
    def add(self, key, value):
        if self.contains(key):
            node = self._find_node(self.root, key)
            for index, k in enumerate(node.keys):
                if k == key:
                    node.values[index] = value
        else:
            super().add(key, value)

### Testing the Init and Add Methods

In [47]:
test_store = KVStore()
assert test_store.split_threshold == 2, "The split is not equal to 2."
test_store.add(1, '1')
test_store.add(1, '2')
assert test_store.get_value(1) == '2', 'The add method did not implement correct.'
test_store.get_value(1)

'2'

## Implementing the Item Getter and Setter

In [57]:
class KVStore(BTree):
    def __init__(self):
        super().__init__(split_threshold=2)
        
    def add(self, key, value):
        if self.contains(key):
            node = self._find_node(self.root, key)
            for index, k in enumerate(node.keys):
                if k == key:
                    node.values[index] = value
        else:
            super().add(key, value)
    
    def __setitem__(self, key, value):
        self.add(key, value)
    
    def __getitem__(self, key):
        return self.get_value(key)
    

### Testing the Getter and Setter Methods

In [62]:
kv = KVStore()
kv[1] = 1
kv[1]

1

In [63]:
kv[1] = 2
kv[1]

2

## Enhancing the Contains Method

In [64]:
class KVStore(BTree):
    def __init__(self):
        super().__init__(split_threshold=2)
        
    def add(self, key, value):
        if self.contains(key):
            node = self._find_node(self.root, key)
            for index, k in enumerate(node.keys):
                if k == key:
                    node.values[index] = value
        else:
            super().add(key, value)
    
    def __setitem__(self, key, value):
        self.add(key, value)
    
    def __getitem__(self, key):
        return self.get_value(key)
    
    def __contains__(self, key):
        return self.contains(key)

### Testing the Contains Method

In [67]:
kv = KVStore()
kv[1] = 1
1 in kv

True

In [68]:
2 in kv

False

## Creating Range Query Methods

In [73]:
class KVStore(BTree):
    def __init__(self):
        super().__init__(split_threshold=2)
        
    def add(self, key, value):
        if self.contains(key):
            node = self._find_node(self.root, key)
            for index, k in enumerate(node.keys):
                if k == key:
                    node.values[index] = value
        else:
            super().add(key, value)
    
    def __setitem__(self, key, value):
        self.add(key, value)
    
    def __getitem__(self, key):
        return self.get_value(key)
    
    def __contains__(self, key):
        return self.contains(key)
    
    def _range_query(self, range_start, range_end, current_node, min_key, max_key):
        if range_start > max_key or range_end < min_key:
            return []
        results = []
        for i, key in enumerate(current_node.keys):
            if range_start <= key and key <= range_end:
                results.append(current_node.values[i])
        if not current_node.is_leaf():
            for i, child in enumerate(current_node.children):
                new_min_key = current_node.keys[i - 1] if i > 0 else min_key
                new_max_key = current_node.keys[i] if i < len(current_node) else max_key
                results += self._range_query(range_start, range_end, child, new_min_key, new_max_key)
        return results 

    def range_query(self, range_start, range_end):
        return self._range_query(range_start, range_end, self.root, float('-inf'), float('inf'))

### Testing the Range Query Method

In [ ]:
class DictKVStore(dict):

    def range_query(self, range_start, range_end):
        result = []
        for key in self.keys():
            if range_start <= key and key <= range_end:
                result.append(self[key])
        return result

In [84]:
kv = KVStore()
dic_store = DictKVStore()
for i in range(100):
    kv[i] = i
    dic_store[i] = i

In [83]:
kv.range_query(3, 5)

[3, 5, 4]

In [85]:
dic_store.range_query(3, 5)

[3, 4, 5]

In [91]:
for range_start, range_end in [(1, 3), (4, 6), (1, 10), (5, 5)]:
    dict_res = sorted(dic_store.range_query(range_start, range_end))
    our_res = sorted(kv.range_query(range_start, range_end))
    assert dict_res == our_res, "Both data structures do not return the same range query result."